# Clustering

**Author: Jessica Cervi**

**Expected time = 2.5 hours**

**Total points = 60 points**
 

## Assignment Overview

In this assignment, we will work on clustering algorithms.

In general, the steps of clustering algorithms are straightforward: 

- The sample points are assigned to centroids 
- The coordinate of the centroid gets updated

These two main steps repeat until we get convergence in our algorithm. When using clustering, some of the difficult steps are:

- How to select the number of centroids required?
- How to initialize the selected centroids?

Although these are important topics in clustering, in this assignment, we will focus on the main steps that define the clustering algorithm. In particular, you will be asked to write functions that compute k-means, soft k-means, and GMM clustering. Like all the other previous assignments, the pre-built versions in the sklearn of these algorithms will be demonstrated at the end of the lesson.

This assignment is designed to build your familiarity and comfort in coding in Python. It will also help you review the key topics from each module. As you progress through the assignment, answers to the questions will get increasingly complex. You must adopt a data scientist's mindset when completing this assignment. Remember to run your code from each cell before submitting your assignment. Running your code beforehand will notify you of errors and giving you a chance to fix your errors before submitting it. You should view your Vocareum submission as if you are delivering a final project to your manager or client. 

***Vocareum Tips***
- Do not add arguments or options to functions unless asked specifically. This will cause an error in Vocareum.
- Do not use a library unless you are explicitly asked in the question. 
- You can download the Grading Report after submitting the assignment. It will include the feedback and hints on incorrect questions. 




### Learning Objectives
- Determine the distance from data through cluster centers
- Assign cluster values using soft/hard clusters  
- Assign cluster values using Gaussian distributions
- Update cluster centers using hard/soft/gaussian mixture clustering 
- Implement Gaussian mixture models and k-means clustering using 'sklearn'


## Index: 

#### Clustering

- [Question 1](#q1)
- [Question 2](#q2)
- [Question 3](#q3)
- [Question 4](#q4)
- [Question 5](#q5)
- [Question 6](#q6)


## Clustering

### Importing the dataset and exploratory data analysis

For this exercise, we will use derived data in two dimensions. Three clusters of data will be derived from three different distributions in four different instances.

We start by examining the data as usual:
- Before coding an algorithm, we will take a look at our data using Python's pandas. For visualizations, we will use matplotlib.

To load the datasets, we will be using the pandas' 'pd.read_csv()' function.

Note that, for this assignment, we have defined the 'path' to each dataset as a string to a variable that gets read by the 'read_csv()' function.

 

In [ ]:
from scipy.stats import multivariate_normal
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

# File Paths
mv_path = "./data/mv.csv"
unif_path = "./data/unif.csv"
mv2_path = "./data/mv2.csv"
mv3_path = "./data/mv3.csv"

# Read in Data
mv_df = pd.read_csv(mv_path, index_col = 0)
unif_df = pd.read_csv(unif_path, index_col = 0)
mv2_df = pd.read_csv(mv2_path, index_col = 0)
mv3_df = pd.read_csv(mv3_path, index_col = 0)

Next, we use 'matplotlib' to visualize the four different sets of distributions. 

In [ ]:
# Create Figure
fig, (axs) = plt.subplots(2,2, figsize = (6,6))

# Plot each group in each dataset as a unique color
for ax, df in zip(axs.flatten(), [mv_df, unif_df, mv2_df, mv3_df]):
    for cat, col in zip(df['cat'].unique(), ["#1b9e77", "#d95f02", "#7570b3"]):
        ax.scatter(df[df.cat == cat].x, df[df.cat == cat].y, c = col, label = cat, alpha = .15)
    ax.legend()

Before we begin with this assignment, we will go through some basic theory of clustering algorithms.

A centroid is defined by a pair of coordinates $(x,y)$. 

In the case of Gaussian mixture models (GMM), the center of each cluster will be initialized using the $2\times2$ covariance matrix  

 $$\Sigma = \begin{bmatrix}
    1 & 0\\
    0&1
\end{bmatrix},$$ 

and parameter $\pi = \frac1k$, with $k$ being the number of clusters.


We will initialize the clusters by using the K-means++ type algorithm.

The K-means++ algorithm chooses one point from the data as the first cluster center. The other cluster centers are chosen from the remaining points. The probability of picking any other point is proportional to the square of the euclidean distance of that point to the nearest cluster center. 

The functions below initialize the centers using K-means.

In [ ]:
# Implement K-means ++ to initialize centers

def pick_cluster_centers(points, num_clusters = 3):
    # Create List to store clusters
    clusters = []
    
    # Save list of cluster indicies
    arr_idx = np.arange(len(points))
    
    # Choose first cluster; append to list
    clusters.append( points[np.random.choice(arr_idx)])
    
    # Define a function to calculate the squared distance
    def dist_sq(x): return np.linalg.norm(x)**2
    
    c_dist = None

    # Add Clusters until reaching "num_clusters"
    while len(clusters) < num_clusters:
        
        # Calculate the distance between latest cluster and rest of points
        new_dist = np.apply_along_axis(np.linalg.norm, 1, points - clusters[-1]).reshape(-1,1)
        
        # Add to distance array - First check to see if the distance matrix exists
        if type(c_dist) == type(None):
            c_dist = new_dist
            
        else:
            c_dist = np.concatenate([c_dist, new_dist], axis = 1)
        
        # Calculate the probability by finding shortest distance, then normalizing
        c_prob = np.apply_along_axis(np.min, 1, c_dist)
        c_prob = c_prob / c_prob.sum()

        # Draw a new cluster according to the probability
        clusters.append(points[np.random.choice(arr_idx, p = c_prob)])
            
    return np.array(clusters)

# Function to add pi and Sigma for GMM clusters
def build_GMM_clusters(clusters):
    return [(c, 1/len(clusters), np.array([[1,0],[0,1]])) for c in clusters]

### Method of Describing Cluster Assignments

In hard clustering, i.e., k-means, each point is assigned exclusively to one cluster. In soft clustering, i.e., soft k-means and GMM, points may be portioned out between clusters.  

For this reason, throughout this assignment, the allocation of the points to the clusters will be stored in a two-dimensional 'NumPy' array. 

For example, assume that we are working with three clusters and one point.
In k-means clustering, if a single point was assigned to cluster '0', its cluster assignment would be '[1,0,0]'. On the other hand, if we are working using soft k-means clustering and the point is evenly split between the three clusters, then its cluster assignment would be '[.33,.33,.33]'.  

In the case of three points and two clusters while using k-means, the cluster-matrix would be: 'np.array([[1,0],[1,0],[0,1]])'. In this case, the first two points are assigned to cluster '0' and the third point is assigned to cluster '1'.  

**Note: the 'np.apply_along_axis' attribute will be useful in the following exercises.**

### Building Cluster Assignment Algorithms

[Back to top](#Index:) 
<a id='q1'></a>

### Question 1:

*10 points*

For each point $i$, the cluster indicator $c_i$ for the cluster $k$ can be computed according to the equation

$$c_i =\text{argmin}_k\ ||x_i - \mu_k||^2.$$

Define a function called 'assign_clusters_k_means' that accepts the following 
arguments:

- A two-dimensional `NumPy` array with the coordinates of each point.
- A two-dimensional `NumPy` array with the coordinates of the centroid of each cluster.

Your function should determine which cluster centroid is closest to each point and return a two-dimensional 'NumPy' array, where each row indicates to which cluster a point is closest to. 

For example, the row [0,1,0,...,0] indicates that the point is assigned to the second cluster, and the row [0,0,...,1] indicates the point is assigned to the last cluster.

In [ ]:
### GRADED

### YOUR SOLUTION HERE
def assign_clusters_k_means(points, clusters):
    """
    Determine the nearest cluster to each point, returning an array indicating the closest cluster
    
    Positional Arguments:
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        clusters: a 2-d NumPy array where each row is a different centroid cluster;
            each column indicates the location of that centroid in that dimension
    
    Example:
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])
        clusters = np.array([[0,1],[5,4]])
        cluster_weights = assign_clusters_k_means(points, clusters)
        
        print(cluster_weights) #--> np.array([[1, 0],
                                              [1, 0],
                                              [0, 1],
                                              [0, 1],
                                              [0, 1]])
    """
    # NB: 'cluster_weights' is used as a common term between functions
    # the name makes more sense in soft-clustering contexts

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q2'></a>

### Question 2:

*10 points*


The proportion $\phi_i(k)$ that a point $i$ belongs to a cluster $k$ is defined by the function for weighted k-means: 

$$\phi_i(k) = \frac{\exp\{-\frac{1}{\beta}||x_i-\mu_k||^2\}}{\sum_j \exp\{-\frac1\beta||x_i-\mu_j||^2\}}.$$ 

Define a  function called 'assign_clusters_soft_k_means' that accepts the following 
arguments:

- A two-dimensional 'NumPy' array with the coordinates of each point.
- A two-dimensional 'NumPy' array with the coordinates of the centroid of each cluster.
- A number 'beta' that defines and helps in identifying the closest distance between the points and clusters.

Your function should return a two-dimensional 'NumPy' array with the cluster weights. Each row in the array should indicate the proportion of how each point 'belongs' to all the clusters. 

For example, if a point 'belongs' evenly to two clusters, the corresponding row in the array should be [.5,.5].  

 


In [ ]:
### GRADED

### YOUR SOLUTION HERE
def assign_clusters_soft_k_means(points, clusters, beta):
    
    """
    Return an array indicating the proportion of the point
        belonging to each cluster
    
    Positional Arguments:
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        clusters: a 2-d NumPy array where each row is a different centroid cluster;
            each column indicates the location of that centroid in that dimension
        beta: a number indicating what distance can be considered 'close'
        
    Example:
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])
        clusters = np.array([[0,1],[5,4]])
        beta = 1
        cluster_weights = assign_clusters_soft_k_means(points, clusters, beta)
        
        print(cluster_weights) #--> np.array([[0.99707331, 0.00292669],
                                              [0.79729666, 0.20270334],
                                              [0.00292669, 0.99707331],
                                              [0.04731194, 0.95268806],
                                              [0.1315826 , 0.8684174 ]])
    """

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q3'></a>

### Question 3:

*10 points*

For this question, the proportion $\phi_i(k)$ that a point $i$ belongs to cluster $k$ is defined by the $\phi$ function for GMM:  

$$\phi_i(k) = \frac{\pi_k N(x_i|\mu_k,\Sigma_k)}{\sum_j \pi_j N(x_i|\mu_j,\Sigma_k)},$$ 

where $N(x_i|\mu_k,\Sigma_k)$ can be computed by using 'stats.multivariate_normal(mu,Sigma).pdf(x)'. For example, if $i = 0$ and $k = 0$, then  

    c = clusters[0]
    p = points[0]
    stats.multivariate_normal(c[0],c[2]).pdf(x)


Define a  function called 'assign_clusters_GMM' that accepts the following 
arguments:

- A two-dimensional 'NumPy' array with the coordinates of each point.
- A list of tuples. The $k^{th}$ tuple in the list should contain the parameters $(\mu_k, \pi_k, \Sigma_k)$ defining the cluster k. In particular, $\mu$ is a one-dimensional 'NumPy' array, $\pi$ is number, and $\Sigma$ is two-dimensional 'NumPy' array.

Your function should return a two-dimensional 'NumPy' array with the cluster weights. Each row in the array should indicate the proportion of how each point 'belongs' to all the clusters. For example, if a point 'belongs' evenly to two clusters, the corresponding row in the array should be [.5,.5].    




In [ ]:
### GRADED
### Follow directions above

### YOUR SOLUTION HERE
def assign_clusters_GMM(points, clusters):
    """    
    Return an array indicating the porportion of the point
        belonging to each cluster
    
    Positional Arguments:
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        clusters: a list of tuples. Each tuple describes a cluster.
            The first element of the tuple is a 1-d NumPy array indicating the
                location of that centroid in each dimension
            The second element of the tuple is a number, indicating the weight (pi)
                of that cluster
            The thrid element is a 2-d NumPy array corresponding to that cluster's
                covariance matrix.
        
    Example:
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])
        clusters = [(np.array([0,1]), 1, np.array([[1,0],[0,1]])),
                    (np.array([5,4]), 1, np.array([[1,0],[0,1]]))]
        
        cluster_weights = assign_clusters_GMM(points, clusters)
        
        print(cluster_weights) #--> np.array([[9.99999959e-01 4.13993755e-08]
                                              [9.82013790e-01 1.79862100e-02]
                                              [4.13993755e-08 9.99999959e-01]
                                              [2.26032430e-06 9.99997740e-01]
                                              [2.47262316e-03 9.97527377e-01]])
    """

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Building Cluster Update Functions

[Back to top](#Index:) 
<a id='q4'></a>

### Question 4:

*10 points*

For this question, the centroid $\mu_k$ for the cluster $k$ should be updated with the formula:  

$$\mu_k = \frac{\sum_ix_i\mathbb{1}\{c_i = k\}}{\sum_i\mathbb{1}\{c_i = k\}},$$

where $c_i$ indicates to which cluster, a point $i$ is assigned. Remember that, generally, the new centroids are the center (mean) of all points associated with the cluster.

Define a  function called 'update_clusters_k_means' that accepts the following 
arguments:

- A two-dimensional 'NumPy' array with the coordinates of each point.
- A two-dimensional 'NumPy' array indicating the cluster assignment to each point. We will define this array to be 'cluster_weights'. Each row of 'cluster_weights' should contain a single 1 and 0's everywhere else. For example, if the point $i$ is assigned to cluster '0', then the row corresponding to the point $i$ in 'cluster_weights' should be [1,0,0,...,0].  

Your function should return a two-dimensional 'NumPy' array giving the new locations of the centroid for each cluster. 

In [ ]:
### GRADED
### Follow directions above

### YOUR SOLUTION HERE
def update_clusters_k_means(points, cluster_weights):
    
    """
    Update the cluster centroids via the k-means algorithm.
    
    Positional Arguments -- 
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        cluster_weights: a 2-d NumPy array where each row corresponds to each row in 'points'
            and the columns indicate which cluster the point 'belongs' to: '1' in the kth
            column indicates belonging to the kth cluster
            
    Example:
    
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])
        cluster_weights = np.array([[1, 0],[1, 0],[0, 1],[0, 1],[0, 1]])
        
        new_cents = update_clusters_k_means(points, cluster_weights)
        
        print(new_cents) #--> np.array([[1. , 1.5],
                                        [4. , 4. ]])
    
    """

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q5'></a>

### Question 5:

*10 points*

For this question, the centroid $\mu_k$ for the cluster $k$ should be updated using the formula:  

$$\mu_k = \frac{\sum_ix_i\phi_i(k)}{\sum_i\phi_i(k)}.$$  

Define a  function called 'update_clusters_soft_k_means' that accepts the following 
arguments:

- A two-dimensional 'NumPy' array with the coordinates of each point.
- A two-dimensional 'NumPy' array indicating the cluster assignment to each point. We will define this array to be 'cluster_weights'. Each row of 'cluster_weights' should contain the numeric weights $\phi_i(k)$ corresponding to the point $i$ and the cluster $k$. The entries of each row should add up to one. For example, if we have two clusters and the point $i$ is evenly assigned to both of them, then the row corresponding to the point $i$ in 'cluster_weights' should be [.5,.5]. Therefore, in this case $\phi_i(0) = .5 \text{ and }\phi_i(1) = .5$.

Your function should return a two-dimensional 'NumPy' array giving the updated locations of the centroid for each cluster. 


In [ ]:
### GRADED
### Follow directions above

### YOUR SOLUTION HERE
def update_clusters_soft_k_means(points, cluster_weights):
    
    """
    Update the cluster centroids according to the soft k-means algorithm
    
    Positional Arguments --
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        cluster_weights: a 2-d NumPy array where each row corresponds to each row in 
            'points'. The values in that row corresponding to the amount that point is associated
            with each cluster.
            
    Example --
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])

        cluster_weights= np.array([[0.99707331, 0.00292669],
                                   [0.79729666, 0.20270334],
                                   [0.00292669, 0.99707331],
                                   [0.04731194, 0.95268806],
                                   [0.1315826 , 0.8684174 ]])
        centroids = update_clusters_soft_k_means(points, cluster_weights)
        
        print(centroids) #--> np. array([[1.15246591, 1.59418291],
                                         [3.87673553, 3.91876291]])
    """

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q6'></a>

### Question 6:

*10 points*

Define a  function called 'update_clusters_GMM' that accepts the following 
arguments:

- A two-dimensional 'NumPy' array with the coordinates of each point.
- A two-dimensional 'NumPy' array indicating the cluster assignment to each point. We will define this array to be 'cluster_weights'. Each row of 'cluster_weights' should contain the numeric weights $\phi_i(k)$ corresponding to the point $i$ and the cluster $k$. The entries of each row should add up to one. For example, if we have two clusters and the point $i$ is evenly assigned to both of them, then the row corresponding to the point $i$ in 'cluster_weights' should be [.5,.5]. 

Your function should return a list of tuples, giving the updated parameters each cluster. In particulat the tuple corresponding to the cluster $k$ will have the format $\mu_k$, $\pi_k$, $\Sigma_k$ where

- $\mu_k$ is a 'NumPy' array with length $d$.  
- $\pi_k$ is a float.  
- $\Sigma_k$ is a 'NumPy' array with dimensions $d\times d$  

For this question, the parameters are updated according to the following formulas:

$$\mu_k = \frac1{n_k}\sum\limits_{i=1}^n \phi_i(k)x_i,\quad  \text{with}\quad  n_k=\sum\limits_{i=1}^n\phi_i(k),$$

$$\pi_k = \frac{n_k}{n},\quad \text{with} \quad n_k=\sum\limits_{i=1}^n\phi_i(k),$$

$$\Sigma_k = \frac1{n_k}\sum\limits_{i=1}^n\phi_i(k)(x_i-\mu_k)(x_i-\mu_k)^T.$$



In [ ]:
### GRADED
### Follow directions above

### YOUR SOLUTION HERE
def update_clusters_GMM(points, cluster_weights):
    
    """
    Update cluster centroids (mu, pi, and Sigma) according to GMM formulas
    
    Positional Arguments --
        points: a 2-d NumPy array where each row is a different point, and each
            column indicates the location of that point in that dimension
        cluster_weights: a 2-d NumPy array where each row corresponds to each row in 
            'points'. the values in that row correspond to the amount that point is associated
            with each cluster.
    Example --
        points = np.array([[0,1], [2,2], [5,4], [3,6], [4,2]])
        cluster_weights = np.array([[9.99999959e-01, 4.13993755e-08],
                                    [9.82013790e-01, 1.79862100e-02],
                                    [4.13993755e-08, 9.99999959e-01],
                                    [2.26032430e-06, 9.99997740e-01],
                                    [2.47262316e-03, 9.97527377e-01]])

        new_clusters = update_clusters_GMM(points, cluster_weights)
        
        print(new_clusters)
            #-->[(array([0.99467691, 1.49609648]), #----> mu, centroid 1
                0.3968977347767351, #-------------------> pi, centroid 1
                array([[1.00994319, 0.50123508],
                       [0.50123508, 0.25000767]])), #---> Sigma, centroid 1
                
                (array([3.98807155, 3.98970927]), #----> mu, centroid 2
                0.603102265479875, #-------------------> pi, centroid 2
                array([[ 0.68695286, -0.63950027], #---> Sigma centroid 2
                       [-0.63950027,  2.67341935]]))]
        
    """

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Clustering in sklearn

Having defined the functions to calculate the cluster assignments and update the centroids, we can now test the clustering functionality on our derived data.  

In the cells below, we define some function to perform the following operations:  

- Create meta-function which combines the 'assign' and 'update' functions into the coherent clustering algorithm with stopping thresholds
- Create k-means, GMM and soft k-means functions
- Create a plotting function to compare 'sklearn' and the custom k-means algorithms  
    - The assignment of initial clusters will be determined by the 'pick_cluster_centers' function defined above, with centers passed explicitly to the custom algorithm and the 'sklearn' algorithm. 

NB: Soft k-means is not implemented in 'sklearn', and the custom GMM function will take too much processing time for Vocareum.

In [ ]:
# Create a function that will create the clustering functions
def create_cluster_func(assign_func, update_func, threshold_func, assign_args = {}):
    
    def cluster(points, centroids, max_iter = 100, stop_threshold = .001):
        for i in range(max_iter):
            old_centroids = centroids
            
            cluster_weights = assign_func(points, centroids, **assign_args)
            centroids = update_func(points, cluster_weights)
            
            if threshold_func(centroids, old_centroids, stop_threshold):
                break
        
        return centroids
    return cluster

# Create functions that test for convergence
def basic_threshold_test(centroids, old_centroids, stop_threshold):
    
    for n, o in zip(centroids, old_centroids):
        #print(n,o)
        if np.linalg.norm(n-o) > stop_threshold:
            return False
    return True

def GMM_threshold_test(centroids, old_centroids, stop_threshold):
    for np, op in zip(centroids, old_centroids):
        if not basic_threshold_test(np,op,stop_threshold):
            return False
    return True

In [ ]:
### Creating the actual functions for clustering
cluster_k_means = create_cluster_func(assign_clusters_k_means,
                                      update_clusters_k_means,
                                      basic_threshold_test)

# Below provided is for experimentation only. 

cluster_GMM = create_cluster_func(assign_clusters_GMM,
                                  update_clusters_GMM,
                                  GMM_threshold_test)


cluster_soft_k_means = create_cluster_func(assign_clusters_soft_k_means,
                                           update_clusters_soft_k_means,
                                           basic_threshold_test,
                                           {'beta':.3}) # This is where the "beta" parameter may be changed

In [ ]:
# Visualization of a function for k-means
from sklearn.cluster import  KMeans


def plot_k_means(clusters = 3):
    # Create figure
    fig, (axs) = plt.subplots(2,2, figsize = (6,6))
    
    # Go thorugh all data sets 
    for ax, df in zip(axs.flatten(), [mv_df, unif_df, mv2_df, mv3_df]):
        
        # Pull out point data
        points = df.iloc[:,:2].values
        
        # Pick random initial centers
        init_cents = pick_cluster_centers(points, clusters)
        
        # Calculate the centers from custom function
        cents = cluster_k_means(points ,init_cents)
        
        # Calculate the centers from sklearn
        km = KMeans(n_clusters= clusters, init = init_cents, n_init=1).fit(points)
        cent_sk = km.cluster_centers_
        
        # Plot each distribution in a different color
        for cat, col in zip(df['cat'].unique(), ["#1b9e77", "#d95f02", "#7570b3"]):
            ax.scatter(df[df.cat == cat].x, df[df.cat == cat].y, label = None, c = col, alpha = .15)
            
        # Plot the Calculated centers
        ax.scatter(cents[:,0], cents[:,1], c = 'k', marker = 'x', label = 'Custom', s = 70)
         # Plot the Calculated centers with sklearn
        ax.scatter(cent_sk[:,0], cent_sk[:,1], c = 'r', marker = '+', label = 'sklearn', s = 70)
        
        # Add legend
        ax.legend()

**NOTE: for faster grading, comment out the code cell below**

In [ ]:
%%time
plot_k_means(3)

### GMM / k-means in 'sklearn'

Clustering algorithms use the common 'fit' and 'predict' syntax of sklearn.  

While the custom algorithms focuses on calculating the cluster centers, the 'prediction' regarding points is implicitly wrapped up in the 'assign' functions. 'sklearn' exposes the functionality more explicitly with '.predict()' methods.

In [ ]:
from sklearn.mixture import GaussianMixture
from matplotlib.colors import to_hex, to_rgb

# Maximum of 8 Clusters(color constraint)
def plot_sk_clust(clusters = 3, clust = GaussianMixture):
    def find_hex(p, colors):
        p = p.reshape(-1,1)
        return to_hex(np.sum(p*colors, axis=0))

    #
    # Create a figure
    fig, (axs) = plt.subplots(2,2, figsize = (6,6))
    
    # Go through all datasets 
    for ax, df in zip(axs.flatten(), [mv_df, unif_df, mv2_df, mv3_df]):
        
        
        # Pull out a point data
        points = df.iloc[:,:2].values
        
        # Give a list of colors as RGB
        colors = ["#1b9e77", "#d95f02", "#7570b3","#e7298a","#66a61e","#e6ab02","#a6761d","#666666"][:clusters]
        colors = [np.array(to_rgb(c)) for c in colors]
        colors = np.array(colors)
        
        
        
        ###
        # SKLEARN 
        ###
        
        # Fit the Clusters from points
        cl = clust(clusters).fit(points)
        
        
        # Calculate the Color values from clusters
        # Note ".predict()" and ".predict_proba()"
        if clust == GaussianMixture:
            probs = cl.predict_proba(points)
            plot_colors = [find_hex(p,colors) for p in probs]
            
        if clust == KMeans:
            probs = cl.predict(points)
            plot_colors = [colors[p] for p in probs]

        ###
        # END SKLEARN
        ###
            
        # Plot the points with colors
        ax.scatter(df.x, df.y, label = None, c = plot_colors, alpha = .15)
    # Add the super title
    fig.suptitle(clust.__str__(clust).split(".")[-1][:-2] + "; Clusters: " + str(clusters))
    plt.show()

#### Visualizations of clustering with `sklearn`

Here's an example of how to visualize clustering using the 'sklearn' module.

In [ ]:
plot_sk_clust(clusters = 3, clust = GaussianMixture)
plot_sk_clust(clusters = 3, clust = KMeans)